In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

sensors_list = [f"sensor_{num}" for num in range(0, 228)]

speeds_df = pd.read_csv("data/PeMSD7_V_228.csv", names=sensors_list)
distances_df = pd.read_csv("data/PeMSD7_W_228.csv", names=range(0, 228))

speeds_df.head()

In [ ]:
# !mkdir data

In [ ]:
distances_df.head()

The information is stored in 228 sensor stations. The distance dataframe is going to be helpful construct the graph since it captures the locations and relative distances of the different sensors. The time series for every sensor is stored in the speeds dataframe, a total of 12672 samples per sensor.

In [ ]:
print(f"Distance data dimensions: {distances_df.shape}")
print(f"Speeds data dimensions: {speeds_df.shape}")

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


fig = px.line(
    speeds_df,
    title="Sensor Time Series",
    labels={
        "value":"Traffic Speed per Sensor",
        "index":"Time in a 5 minutes interval"
    },

)
fig.show()

In [ ]:
mean_speed = speeds_df.mean(axis=1)


fig = px.line(
    mean_speed,
    title="Mean Speed Time Series",
    labels={
        "value":"Mean Speed",
        "index":"Time in a 5 minutes interval"
    },

)
fig.show()


In [ ]:
distances_df.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


fig = px.imshow(
    distances_df,
    color_continuous_scale=px.colors.sequential.ice,
    title="Distance Sensor Data"
)
fig.show()

Data Processing

In [ ]:
def compute_adjacency_matrix(distances, sigma_squared=0.1, epsilon=0.5):
    distances_normalized_array = distances.to_numpy() / 10000.
    distance_squared = distances_normalized_array * distances_normalized_array
    n = distance_squared.shape[0]
    w_mask = np.ones([n, n]) - np.identity(n)
    return np.exp(-distance_squared / sigma_squared) * (np.exp(-distance_squared / sigma_squared) >= epsilon) * w_mask

adj = compute_adjacency_matrix(distances_df)

In [ ]:
fig = px.imshow(
    adj,
    color_continuous_scale=px.colors.sequential.ice,
    title="Adjacency Matrix"
)
fig.show()

In [ ]:
import networkx as nx

rows, cols = np.where(adj > 0)
edges = zip(rows.tolist(), cols.tolist())

G = nx.Graph()
G.add_edges_from(edges)
G.nodes

In [ ]:
len(list(G.nodes))

In [ ]:
list(G.edges)[:10]

In [ ]:
node_position = nx.spring_layout(G)
print("Number of Nodes")
print(len(node_position))
print("Nodes")
print(node_position.keys())
print("Positions")
print(node_position.values())

In [ ]:
for node in G.nodes:
  G.nodes[node]['pos'] = node_position[node]

In [ ]:
G.nodes[0]

In [ ]:


edge_x = []
edge_y = []
for edge in G.edges():
    first_node = edge[0]
    second_node = edge[1]
    x0, y0 = G.nodes[first_node]["pos"]
    x1, y1 = G.nodes[second_node]["pos"]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')



node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)




node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))


node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text


fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Traffic Sensor Data',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="",#"Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# Apply z-score
def zscore(x, mean, std):
    return (x - mean) / std

speeds_norm = zscore(speeds_df, speeds_df.mean(axis=0), speeds_df.std(axis=0))

# Create dataset
lags = 24
horizon = 48
xs = []
ys = []
for i in range(lags, speeds_norm.shape[0]-horizon):
    xs.append(speeds_norm.to_numpy()[i-lags:i].T)
    ys.append(speeds_norm.to_numpy()[i+horizon-1])

# Convert adjacency matrix to edge_index (COO format)
edge_index = (np.array(adj) > 0).nonzero()
edge_index

In [ ]:
!pip install -q torch-scatter~=2.1.0 torch-sparse~=0.6.16 torch-cluster~=1.6.0 torch-spline-conv~=1.2.1 torch-geometric==2.2.0 -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install -q torch-geometric-temporal==0.54.0

import torch
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
from torch_geometric_temporal.signal import temporal_signal_split


dataset = StaticGraphTemporalSignal(edge_index, adj[adj > 0], xs, ys)
dataset[0]

In [ ]:

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

In [ ]:
import torch
from torch_geometric_temporal.nn.recurrent import A3TGCN


class TemporalGNN(torch.nn.Module):
    def __init__(self, dim_in, periods):
        super().__init__()
        self.tgnn = A3TGCN(in_channels=dim_in, out_channels=32, periods=periods)
        self.linear = torch.nn.Linear(32, periods)

    def forward(self, x, edge_index, edge_attr):
        h = self.tgnn(x, edge_index, edge_attr).relu()
        h = self.linear(h)
        return h

model = TemporalGNN(lags, 1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
model.train()
print(model)

In [ ]:
from tqdm import tqdm
num_epochs = 50
# Training
for epoch in range(1, num_epochs + 1):
    loss = 0
    step = 0
    for i, snapshot in enumerate(train_dataset):
        y_pred = model(snapshot.x.unsqueeze(2), snapshot.edge_index, snapshot.edge_attr)
        loss += torch.mean((y_pred-snapshot.y)**2)
        step += 1
    loss = loss / (step + 1)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if epoch % 5 == 0:
        print(f"Epoch {epoch:>2} | Train MSE: {loss:.4f}")

In [ ]:
def inverse_zscore(x, mean, std):
    return x * std + mean

y_test = []
for snapshot in test_dataset:
    y_hat = snapshot.y.numpy()
    y_hat = inverse_zscore(y_hat, speeds_df.mean(axis=0), speeds_df.std(axis=0))
    y_test = np.append(y_test, y_hat)

gnn_pred = []
model.eval()
for snapshot in test_dataset:
    snapshot = snapshot
    y_hat = model(snapshot.x.unsqueeze(2), snapshot.edge_index, snapshot.edge_weight).squeeze().detach().numpy()
    y_hat = inverse_zscore(y_hat, speeds_df.mean(axis=0), speeds_df.std(axis=0))
    gnn_pred = np.append(gnn_pred, y_hat)



def MAE(real, pred):
    return np.mean(np.abs(pred - real))

def RMSE(real, pred):
    return np.sqrt(np.mean((pred - real) ** 2))

def MAPE(real, pred):
    return np.mean(np.abs(pred - real) / (real + 1e-5))

print(f'GNN MAE  = {MAE(gnn_pred, y_test):.4f}')
print(f'GNN RMSE = {RMSE(gnn_pred, y_test):.4f}')
print(f'GNN MAPE = {MAPE(gnn_pred, y_test):.4f}')

In [ ]:

rw_pred = []
for snapshot in test_dataset:
    y_hat = snapshot.x[:,-1].squeeze().detach().numpy()
    y_hat = inverse_zscore(y_hat, speeds_df.mean(axis=0), speeds_df.std(axis=0))
    rw_pred = np.append(rw_pred, y_hat)

print(f'RW MAE  = {MAE(rw_pred, y_test):.4f}')
print(f'RW RMSE = {RMSE(rw_pred, y_test):.4f}')
print(f'RW MAPE = {MAPE(rw_pred, y_test):.4f}')



In [ ]:
ha_pred = []
for i in range(lags, speeds_norm.shape[0]-horizon):
    y_hat = speeds_norm.to_numpy()[:i].T.mean(axis=1)
    y_hat = inverse_zscore(y_hat, speeds_df.mean(axis=0), speeds_df.std(axis=0))
    ha_pred.append(y_hat)
ha_pred = np.array(ha_pred).flatten()[-len(y_test):]

print(f'HA MAE  = {MAE(ha_pred, y_test):.4f}')
print(f'HA RMSE = {RMSE(ha_pred, y_test):.4f}')
print(f'HA MAPE = {MAPE(ha_pred, y_test):.4f}')

In [ ]:
y_preds = [inverse_zscore(model(snapshot.x.unsqueeze(2), snapshot.edge_index, snapshot.edge_weight).squeeze().detach().numpy(), speeds_df.mean(axis=0), speeds_df.std(axis=0)).mean() for snapshot in test_dataset]

mean = speeds_df.mean(axis=1)
std = speeds_df.std(axis=1)


In [ ]:
!mkdir models

In [ ]:
PATH = "models/traffic_forecast.pt"
torch.save(model, PATH)